# Sistemas inteligentes para respostas a perguntas médicas

Gyovana M. Moriyama (216190)

Rafael A. Matumoto (273085)

In [ ]:
!pip install -qU langchain_openai langchain-community faiss-cpu sentence-transformers openai datasets pydantic langgraph

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.4/50.4 kB 3.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 29.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 27.5/27.5 MB 37.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 389.3/389.3 kB 21.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 20.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 119.3/119.3 kB 5.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 9.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 11.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 45.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 408.7/408.7 kB 23.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 8.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 56.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/

## ReAct

In [ ]:
import os
import datasets
import datetime
import re

from google.colab import userdata, drive

from tqdm import tqdm
from pydantic import BaseModel, Field
from typing import List, Optional, Literal

from langchain.text_splitter import CharacterTextSplitter, RecursiveCharacterTextSplitter
from langchain_community.document_loaders import TextLoader, DirectoryLoader
from langchain.vectorstores import FAISS
from langchain.embeddings import HuggingFaceBgeEmbeddings

from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate
from langchain.agents import AgentExecutor, create_react_agent
from langchain.tools.retriever import create_retriever_tool
from langchain_core.load import dumps, loads

In [ ]:
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
os.environ['OPENAI_API_KEY'] = userdata.get('api_key')

### MedQA-USMLE-4-options dataset

In [ ]:
data = datasets.load_dataset('GBaker/MedQA-USMLE-4-options', split='test')

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md:   0%|          | 0.00/654 [00:00<?, ?B/s]

phrases_no_exclude_train.jsonl:   0%|          | 0.00/16.2M [00:00<?, ?B/s]

phrases_no_exclude_test.jsonl:   0%|          | 0.00/2.08M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/10178 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/1273 [00:00<?, ? examples/s]

In [ ]:
sampled_question = data[0]

### Load saved retrievers

In [ ]:
emb_model_name = 'sentence-transformers/all-MiniLM-L6-v2'
embedding_model = HuggingFaceBgeEmbeddings(model_name=emb_model_name)

The cache for model files in Transformers v4.22.0 has been updated. Migrating your old cache. This is a one-time only operation. You can interrupt this and resume the migration later on by calling `transformers.utils.move_cache()`.


0it [00:00, ?it/s]

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.7k [00:00<?, ?B/s]

...

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [ ]:
# load saved faiss indexes
faiss = FAISS.load_local(
    folder_path='/content/drive/MyDrive/ia024a/projeto/entrega3/retriever/faiss__500_50__20241103_135850',
    embeddings=embedding_model,
    allow_dangerous_deserialization=True
)

In [ ]:
# create a search tool from the faiss retriever
tool_search_faiss = create_retriever_tool(
    retriever=faiss.as_retriever(),
    name='search_docs',
    description='Searchs the query in documents and returns the top-3 most relevant ones.',
)

In [ ]:
print(tool_search_faiss.invoke('meconium ileus'))

emesis. Abdominal radiographs show dilated loops of intestine. Because the enteric contents are so viscous, air-fluid levels do not form, even when obstruction is complete. Small bubbles of gas become entrapped in the inspissated meconium in the dis-tal ileum, where they produce a characteristic “ground glass” appearance.The diagnosis of meconium ileus is confirmed by a con-trast enema that typically demonstrates a microcolon. In patients with uncomplicated meconium ileus, the terminal ileum is

The ileal branch passes to the left and ascends to supply the final part of the ileum before anastomosing with the superior mesenteric artery.

The arterial supply to the ileum (Fig. 4.71) includes: ileal arteries from the superior mesenteric artery, and an ileal branch from the ileocolic artery (from the superior mesenteric artery).

The large intestine extends from the distal end of the ileum to the anus, a distance of approximately 1.5 m in adults. It absorbs fluids and salts from the gut co

### ReAct agent

In [ ]:
model = 'gpt-4o-mini'
model_temp = 0.5

In [ ]:
llm = ChatOpenAI(
    model=model,
    temperature=model_temp,
)

In [ ]:
# react prompt
react_prompt_template = '''
Solve the question answering task alternating between Question, Thought, Action, Input and Observation steps.
You only have access to the following tools: {tools}

Use the following format:
Question: the input question you must answer
Thought: you should always think about what to do. Break the problem down into subproblems and smaller steps and decide which action to take.
Action: the action to take, must be one of [{tool_names}]. If no action is needed, return your Final answer instead.
Action Input: the input to the action
Observation: the output of the action.
... (this Thought/Action/Action Input/Observation can repeat any number of times)
Final Answer: A/B/C/D

Begin!
Question: {question}
Options: {options}
Thought: {agent_scratchpad}
'''.strip()

In [ ]:
react_prompt = ChatPromptTemplate.from_template(react_prompt_template)

### ReAct with FAISS retriever

https://python.langchain.com/docs/how_to/serialization/

In [ ]:
agent = create_react_agent(
    llm=llm,
    tools=[tool_search_faiss],
    prompt=react_prompt
)

In [ ]:
agent_executor = AgentExecutor(
    agent=agent,
    tools=[tool_search_faiss],
    verbose=False,
    handle_parsing_errors=True,
    return_intermediate_steps=True,
    max_iterations=5
)

In [ ]:
res = agent_executor.invoke({'question': sampled_question['question'], 'options': sampled_question['options']})

In [ ]:
res

{'question': 'A junior orthopaedic surgery resident is completing a carpal tunnel repair with the department chairman as the attending physician. During the case, the resident inadvertently cuts a flexor tendon. The tendon is repaired without complication. The attending tells the resident that the patient will do fine, and there is no need to report this minor complication that will not harm the patient, as he does not want to make the patient worry unnecessarily. He tells the resident to leave this complication out of the operative report. Which of the following is the correct next action for the resident to take?',
 'options': {'A': 'Disclose the error to the patient and put it in the operative report',
  'B': 'Tell the attending that he cannot fail to disclose this mistake',
  'C': 'Report the physician to the ethics committee',
  'D': 'Refuse to dictate the operative report'},
 'output': 'A',
 'intermediate_steps': [(AgentAction(tool='search_docs', tool_input='medical ethics disclo

In [ ]:
res['intermediate_steps']

[(AgentAction(tool='search_docs', tool_input='medical ethics disclosure surgical error', log='The situation involves an ethical dilemma regarding the disclosure of a surgical error. The options provided suggest different approaches to handling the attending physician\'s instruction to omit the complication from the operative report. I need to evaluate the implications of each option in terms of ethical standards, patient safety, and professional responsibilities.\n\n1. **Option A**: Disclosing the error to the patient and including it in the operative report aligns with ethical principles of honesty and transparency in medicine. However, it may conflict with the attending\'s wishes.\n  \n2. **Option B**: Telling the attending that he cannot fail to disclose this mistake shows a commitment to ethical standards, but it may create tension in the resident-attending relationship.\n\n3. **Option C**: Reporting the physician to the ethics committee could be seen as an extreme measure and may 

In [ ]:
print(res['intermediate_steps'][0][0].log)

The situation involves an ethical dilemma regarding the disclosure of a surgical error. The options provided suggest different approaches to handling the attending physician's instruction to omit the complication from the operative report. I need to evaluate the implications of each option in terms of ethical standards, patient safety, and professional responsibilities.

1. **Option A**: Disclosing the error to the patient and including it in the operative report aligns with ethical principles of honesty and transparency in medicine. However, it may conflict with the attending's wishes.
  
2. **Option B**: Telling the attending that he cannot fail to disclose this mistake shows a commitment to ethical standards, but it may create tension in the resident-attending relationship.

3. **Option C**: Reporting the physician to the ethics committee could be seen as an extreme measure and may not be the most appropriate first step.

4. **Option D**: Refusing to dictate the operative report cou

In [ ]:
print(res['intermediate_steps'][0][1])

point out several fallacies about disclosure, including that disclosing medical error always leads to litigation and that error always means negligence.

regarding a medical error may be needed so that patients can make independent and well-informed deci-sions about future aspects of their care. The principles of auton-omy and justice dictate that surgeons need to respect individuals by being fair in providing accurate information about all aspects of their care—even when an error has occurred.Disclosing one’s own errors is therefore part of the ethi-cal standard of honesty and putting the patient’s interests above one’s own. Disclosing the

Three programs are worth noting in any discussion of disclosure and apology for medical error. First is the University of Michigan’s patient safety program, which addresses the need to disclose medical error in several publications (57). Important points are made about a patient’s rights concerning disclosure of medical error and that an apology fo

In [ ]:
res['output']

'A'

In [ ]:
with open(f'/content/drive/MyDrive/ia024a/projeto/entrega3/results/answers_faiss_{datetime.datetime.now().strftime("%Y%m%d_%H%M%S")}.txt', 'w') as f:

    f.write(''.center(10, '-'))
    f.write('\n')
    f.write(f'Model: {model}\n')
    f.write(f'Temperature: {model_temp}\n')
    f.write(f'Prompt: {react_prompt}\n')
    f.write(''.center(10, '-'))
    f.write('\n[{')

    for n, question in enumerate(tqdm(data)):

        # if n == 49:
        #     break

        res = agent_executor.invoke({'question': question['question'], 'options': question['options']})
        f.write(f'"{n}": ')

        # serialize results as a string
        f.write(dumps(res))
        f.write(',')

    f.write('"placeholder": "placeholder"') # add a placeholder k:v pair for trailling comma above
    f.write('}]')

100%|██████████| 1273/1273 [2:16:22<00:00,  6.43s/it]


#### Results FAISS

In [ ]:
os.listdir('/content/drive/MyDrive/ia024a/projeto/entrega3/results')

['answers_faiss_20241103_151837.txt',
 'answers_faiss_20241103_152229.txt',
 'answers_faiss_20241103_152940.txt',
 'answers_faiss_20241103_153308.txt',
 'answers_faiss_20241103_153501.txt',
 'answers_faiss_20241103_154054.txt',
 'answers_faiss_20241103_155219.txt',
 'answers_faiss_20241103_165217.txt']

In [ ]:
# load log file with LLM results
with open(f'/content/drive/MyDrive/ia024a/projeto/entrega3/results/answers_faiss_20241103_165217.txt') as f:
    tmp = f.readlines()

In [ ]:
# recover results as a dictionary
recovered_results = loads(tmp[-1])[0]

In [ ]:
# 1273 questions + placeholder key
len(recovered_results)

1274

In [ ]:
# check outputs
for n, (k, v) in enumerate(recovered_results.items()):

    if k == 'placeholder':
        continue

    print(k, v['output'], data[n]['answer_idx'])

0 A B
1 D D
2 B B
3 D D
4 B B
5 B D
6 B C
7 C C
8 B B
9 A A
10 D D
11 D D
12 B B
13 D D
14 Agent stopped due to iteration limit or time limit. C
15 A B
16 D D
17 D D
18 B B
19 B D
20 C C
21 A A
22 C C
23 Agent stopped due to iteration limit or time limit. D
24 A A
25 D D
26 D D
27 A A
28 D D
29 C C
30 D D
31 D C
32 D D
33 D B
34 D A
35 A A
36 C D
37 A A
38 B D
39 A A
40 C C
41 C C
42 A A
43 A B
44 C A
45 A D
46 C C
47 Agent stopped due to iteration limit or time limit. A
48 Agent stopped due to iteration limit or time limit. B
49 C C
50 D D
51 A A
52 D D
53 C C
54 A A
55 A A
56 C C
57 B B
58 C C
59 C A
60 B C
61 A A
62 A D
63 A A
64 C A
65 B B
66 C C
67 B B
68 D D
69 C A
70 A A
71 A A
72 D C
73 B B
74 C C
75 A A
76 B B
77 C B
78 C C
79 C C
80 B B
81 C A
82 C C
83 A A
84 C C
85 B B
86 C C
87 C C
88 C C
89 B B
90 D D
91 B B
92 C C
93 C D
94 C C
95 D D
96 A A
97 B B
98 B B
99 B B
100 D D
101 A A
102 A A
103 A A
104 B B
105 B B
106 A B
107 A A
108 D D
109 D B
110 C C
111 B B
112 A B
113 C 

In [ ]:
limit_error = list()
not_parsed = list()
answers = list()

# matches literals A, B, C, or D at the beginning of a sentence (followed by the text of the corresponding option)
# e.g. A (answer text) => A
# other cases will be ignored
ans_ptn = re.compile(r'(^[A|B|C|D])\s\(')

for n, (k, v) in enumerate(recovered_results.items()):

    # skip placeholder key
    if k == 'placeholder':
        continue

    # max_iteration set to 5
    if v['output'] == 'Agent stopped due to iteration limit or time limit.':
        answers.append(None)
        limit_error.append(k)

    elif len(v['output']) > 1:
        tmp_output = ans_ptn.findall(v['output'])
        if len(tmp_output) == 1:
            answers.append(tmp_output[0])
        elif len(tmp_output) > 1:
            answers.append(None)
            not_parsed.append(k)
            print(tmp_output)
        else:
            answers.append(None)
            not_parsed.append(k)
            print(k, v['output'])

    else:
        if v['output'] in ['A', 'B', 'C', 'D']:
            answers.append(v['output'])
        else:
            answers.append(None)

226 A/C/D
798 C/D
869 ** C
1161 None of the above options accurately represent the factor provided to the patient.


In [ ]:
# unanswered questions or answers not properly parsed
len(limit_error)

53

In [ ]:
# parsing errors/other errors
len(not_parsed)

4

In [ ]:
# result_dict[question]['intermediate_steps'] -> List[List[AgentAction(tool, tool_input, log), response: str]]
def format_intermediate_step(answer):

    formatted_output = ''
    for i in answer['intermediate_steps']:
        formatted_output += i[0].log
        formatted_output += '\n\n'
        formatted_output += i[1]
        formatted_output += '\n'

    return formatted_output

In [ ]:
react_logs = list()

for i in recovered_results.values():
    if i == 'placeholder':
        continue
    react_logs.append(format_intermediate_step(i))

In [ ]:
data = data.add_column('answer_react_faiss', answers)
data = data.add_column('react_log', react_logs)

In [ ]:
data.save_to_disk(f'/content/drive/MyDrive/ia024a/projeto/entrega3/results/dataset_faiss_500_50')

Saving the dataset (0/1 shards):   0%|          | 0/1273 [00:00<?, ? examples/s]

In [ ]:
sum([pred == gt for gt, pred in zip(data['answer_idx'], answers)]) / len(data)

0.7344854673998429

In [ ]:
df = data.to_pandas()

In [ ]:
df.to_csv(f'/content/drive/MyDrive/ia024a/projeto/entrega3/results/dataset_faiss_500_50.csv')

In [ ]:
# react with faiss accuracy
accuracy = df['answer_react_faiss'].eq(df['answer_idx']).mean()
accuracy

0.7344854673998429

In [ ]:
# react accuracy for step 1 questions
step1 = df[df['meta_info'].eq('step1')]

step1_acc = step1['answer_react_faiss'].eq(step1['answer_idx']).mean()
step1_acc

0.7378497790868925

In [ ]:
# react accuracy for step 2&3 questions
step23 = df[df['meta_info'].eq('step2&3')]

step23_acc = step23['answer_react_faiss'].eq(step23['answer_idx']).mean()
step23_acc

0.7306397306397306

In [ ]:
df[df['answer_react_faiss'].eq(df['answer_idx'])]

,question,answer,options,meta_info,answer_idx,metamap_phrases,answer_react_faiss,react_log
1,A 67-year-old man with transitional cell carci...,Cross-linking of DNA,"{'A': 'Inhibition of proteasome', 'B': 'Hypers...",step1,D,"[67 year old man, transitional cell carcinoma ...",D,"To answer the question, I need to analyze the ..."
2,Two weeks after undergoing an emergency cardia...,Cholesterol embolization,"{'A': 'Renal papillary necrosis', 'B': 'Choles...",step2&3,B,"[Two weeks, emergency cardiac, stenting, unsta...",B,"To answer the question, I need to analyze the ..."
3,A 39-year-old woman is brought to the emergenc...,"Lactose-fermenting, gram-negative rods forming...","{'A': 'Coagulase-positive, gram-positive cocci...",step1,D,"[year old woman, brought, emergency department...",D,"To solve this question, I need to analyze the ..."
4,A 35-year-old man comes to the physician becau...,Ketotifen eye drops,"{'A': 'Erythromycin ointment', 'B': 'Ketotifen...",step2&3,B,"[35 year old man, physician, of itchy, watery,...",B,"To solve this question, I need to analyze the ..."
7,A 65-year-old man is brought to the emergency ...,Clopidogrel,"{'A': 'Nifedipine', 'B': 'Enoxaparin', 'C': 'C...",step2&3,C,"[65 year old man, brought, emergency departmen...",C,"To solve this question, I need to analyze the ..."
...,...,...,...,...,...,...,...,...
1265,A 74-year-old woman with a history of hyperten...,Delirium,"{'A': 'Alzheimer dementia', 'B': 'Delirium', '...",step1,B,"[74 year old woman, history of hypertension, h...",B,To determine the most likely diagnosis for the...
1266,A 45-year-old male with a history of diabetes ...,Nifedipine,"{'A': 'Metoprolol', 'B': 'Verapamil', 'C': 'Ni...",step1,C,"[year old male, history, diabetes, poorly cont...",C,"To answer the question, I need to analyze the ..."
1268,A 39-year-old woman presents to the clinic for...,Thyroid-stimulating hormone (TSH),"{'A': 'Glucose', 'B': 'Triiodothyronine (T3)',...",step2&3,D,"[year old woman presents, clinic, annual physi...",D,"To answer this question, I need to analyze the..."
1271,A 19-year-old man is brought to the physician ...,Schizotypal personality disorder,"{'A': 'Social anxiety disorder', 'B': 'Avoidan...",step2&3,C,"[year old man, brought, physician, mother, wor...",C,To determine the most likely diagnosis for the...


In [ ]:
# distribution of wrong answers
df[~df['answer_react_faiss'].eq(df['answer_idx'])].groupby('answer_react_faiss').size()

,0
answer_react_faiss,
A,71
B,77
C,67
D,66


In [ ]:
# sample of correctly answered question
sample_answer = df[df['answer_react_faiss'].eq(df['answer_idx'])].sample(1)

In [ ]:
print(sample_answer['question'].values[0])

A 38-year-old woman undergoes a partial thyroidectomy for an enlarged thyroid gland compressing the trachea. During surgery, a young surgeon ligates the inferior thyroid arteries as close to the poles as possible to prevent intraoperative hemorrhage. 2 weeks later, the patient presents to the outpatient clinic with hoarseness. What could be the most likely cause of her hoarseness?


In [ ]:
print(sample_answer['options'].values[0])

{'A': 'Internal laryngeal nerve injury', 'B': 'Recurrent laryngeal nerve injury', 'C': 'External laryngeal nerve injury', 'D': 'Laryngeal edema'}


In [ ]:
print(sample_answer['answer_react_faiss'].values[0])

B


In [ ]:
print(sample_answer['react_log'].values[0])

To solve this question, I need to analyze the surgical procedure and its potential complications. The patient underwent a partial thyroidectomy, and the hoarseness that developed two weeks later could be attributed to nerve injuries that can occur during thyroid surgeries. 

1. The recurrent laryngeal nerve is particularly at risk during thyroid surgeries, as it runs near the thyroid gland and is responsible for innervating most of the intrinsic muscles of the larynx, which control voice. Injury to this nerve commonly results in hoarseness.
  
2. The internal laryngeal nerve primarily provides sensory innervation above the vocal cords and is less likely to cause hoarseness if injured.

3. The external laryngeal nerve innervates the cricothyroid muscle, which helps in pitch modulation, but its injury would not typically present with hoarseness.

4. Laryngeal edema could cause hoarseness but is usually associated with more immediate post-operative complications rather than a delayed pres